In [1]:
# !pip install gymnasium[classic_control] torch numpy wandb moviepy matplotlib seaborn

In [2]:
import gymnasium as gym
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import wandb
import random
import os
from collections import deque
from torch.distributions import Normal, Categorical
import os

os.environ["XDG_RUNTIME_DIR"] = "/tmp/runtime-root"
os.makedirs("/tmp/runtime-root", exist_ok=True)
# Ensure GPU is used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Running on device: {device}")

# Create directories for artifacts
os.makedirs("videos", exist_ok=True)
os.makedirs("models", exist_ok=True)

✅ Running on device: cuda


In [3]:
class RolloutBuffer:
    """For On-Policy Agents (PPO, A2C)"""
    def __init__(self):
        self.actions = []
        self.states = []
        self.logprobs = []
        self.rewards = []
        self.dones = []
    
    def clear(self):
        del self.actions[:]
        del self.states[:]
        del self.logprobs[:]
        del self.rewards[:]
        del self.dones[:]

class ReplayBuffer:
    """For Off-Policy Agents (SAC)"""
    def __init__(self, capacity, state_dim, action_dim):
        self.capacity = capacity
        self.buffer = deque(maxlen=capacity)
    
    def push(self, state, action, reward, next_state, done):
        self.buffer.append((state, action, reward, next_state, done))
    
    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        state, action, reward, next_state, done = map(np.stack, zip(*batch))
        return (
            torch.FloatTensor(state).to(device),
            torch.FloatTensor(action).to(device),
            torch.FloatTensor(reward).unsqueeze(1).to(device),
            torch.FloatTensor(next_state).to(device),
            torch.FloatTensor(done).unsqueeze(1).to(device)
        )
    
    def __len__(self):
        return len(self.buffer)

In [ ]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.orthogonal_(m.weight, gain=np.sqrt(2))
        nn.init.constant_(m.bias, 0)

class ActorCritic(nn.Module):
    """Shared architecture for PPO and A2C"""
    def __init__(self, state_dim, action_dim, is_continuous=False):
        super(ActorCritic, self).__init__()
        self.is_continuous = is_continuous
        
        # Shared Feature Extractor
        self.base = nn.Sequential(
            nn.Linear(state_dim, 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Tanh()
        )
        
        # Actor Heads
        # Outputs Mean and LogStd for Continuous (e.g. Pendulum): Mean=[0.5], Std=exp(LogStd)=[0.2].
        if is_continuous:
            self.actor_mean = nn.Linear(64, action_dim)
            self.actor_logstd = nn.Parameter(torch.zeros(1, action_dim)) # Learnable std
        else: # Outputs Logits. Softmax applied in Categorical. e.g. (CartPole): [0.1, 0.9] (10% Left, 90% Right).
            self.actor = nn.Linear(64, action_dim)

        # Critic Head
        self.critic = nn.Linear(64, 1)
        self.apply(init_weights)

    def forward(self):
        raise NotImplementedError
    
    def act(self, state):
        x = self.base(state)
        
        if self.is_continuous:
            mean = self.actor_mean(x)
            std = self.actor_logstd.exp().expand_as(mean)
            dist = Normal(mean, std)
        else:
            logits = self.actor(x)
            dist = Categorical(logits=logits)

        action = dist.sample()
        action_logprob = dist.log_prob(action)
        
        if self.is_continuous:
            action_logprob = action_logprob.sum(dim=-1) # Sum logprobs for multidim actions
            
        return action.detach(), action_logprob.detach()
    
    def evaluate(self, state, action):
        x = self.base(state)
        value = self.critic(x)
        
        if self.is_continuous:
            mean = self.actor_mean(x)
            std = self.actor_logstd.exp().expand_as(mean)
            dist = Normal(mean, std)
            action_logprobs = dist.log_prob(action).sum(dim=-1)
            dist_entropy = dist.entropy().sum(dim=-1)
        else:
            logits = self.actor(x)
            dist = Categorical(logits=logits)
            action_logprobs = dist.log_prob(action)
            dist_entropy = dist.entropy()
            
        return action_logprobs, value, dist_entropy

In [5]:
class PPOAgent:
    def __init__(self, state_dim, action_dim, lr, gamma, k_epochs, eps_clip, is_continuous):
        self.policy = ActorCritic(state_dim, action_dim, is_continuous).to(device)
        self.optimizer = optim.Adam(self.policy.parameters(), lr=lr)
        self.policy_old = ActorCritic(state_dim, action_dim, is_continuous).to(device)
        self.policy_old.load_state_dict(self.policy.state_dict())
        
        self.gamma = gamma
        self.k_epochs = k_epochs
        self.eps_clip = eps_clip
        self.buffer = RolloutBuffer()
        self.loss_fn = nn.MSELoss()
        self.is_continuous = is_continuous

    def select_action(self, state):
        with torch.no_grad():
            state = torch.FloatTensor(state).unsqueeze(0).to(device)
            action, action_logprob = self.policy_old.act(state)
            
        self.buffer.states.append(state.squeeze(0)) # Store as tensor
        self.buffer.actions.append(action.squeeze(0))
        self.buffer.logprobs.append(action_logprob.squeeze(0))
        
        if self.is_continuous:
            return action.detach().cpu().numpy().flatten()
        return action.item()

    def update(self):
        # Calculate Rewards (Monte Carlo)
        rewards = []
        discounted_reward = 0
        for reward, is_terminal in zip(reversed(self.buffer.rewards), reversed(self.buffer.dones)):
            if is_terminal:
                discounted_reward = 0
            discounted_reward = reward + (self.gamma * discounted_reward)
            rewards.insert(0, discounted_reward)
            
        rewards = torch.tensor(rewards, dtype=torch.float32).to(device)
        rewards = (rewards - rewards.mean()) / (rewards.std() + 1e-7) # Normalize

        # Stack Buffer
        old_states = torch.stack(self.buffer.states).detach().to(device)
        old_actions = torch.stack(self.buffer.actions).detach().to(device)
        old_logprobs = torch.stack(self.buffer.logprobs).detach().to(device)

        # Optimize
        for _ in range(self.k_epochs):
            logprobs, state_values, dist_entropy = self.policy.evaluate(old_states, old_actions)
            state_values = torch.squeeze(state_values)
            
            # Ratios and Advantages
            ratios = torch.exp(logprobs - old_logprobs)
            advantages = rewards - state_values.detach()
            
            # PPO Loss 
            surr1 = ratios * advantages
            surr2 = torch.clamp(ratios, 1 - self.eps_clip, 1 + self.eps_clip) * advantages
            
            loss = -torch.min(surr1, surr2) + 0.5 * self.loss_fn(state_values, rewards) - 0.01 * dist_entropy
            
            self.optimizer.zero_grad()
            loss.mean().backward()
            self.optimizer.step()
            
        self.policy_old.load_state_dict(self.policy.state_dict())
        self.buffer.clear()

class A2CAgent(PPOAgent):
    """A2C is strictly single-step update without clipping."""
    def update(self):
        # Same Reward Logic
        rewards = []
        discounted_reward = 0
        for reward, is_terminal in zip(reversed(self.buffer.rewards), reversed(self.buffer.dones)):
            if is_terminal: discounted_reward = 0
            discounted_reward = reward + (self.gamma * discounted_reward)
            rewards.insert(0, discounted_reward)
        rewards = torch.tensor(rewards, dtype=torch.float32).to(device)
        rewards = (rewards - rewards.mean()) / (rewards.std() + 1e-7)

        old_states = torch.stack(self.buffer.states).detach().to(device)
        old_actions = torch.stack(self.buffer.actions).detach().to(device)

        # Single pass, no clip
        logprobs, state_values, dist_entropy = self.policy.evaluate(old_states, old_actions)
        state_values = torch.squeeze(state_values)
        
        advantages = rewards - state_values
        actor_loss = -(logprobs * advantages.detach()).mean()
        critic_loss = self.loss_fn(state_values, rewards)
        
        loss = actor_loss + 0.5 * critic_loss - 0.001 * dist_entropy.mean()
        
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        self.policy_old.load_state_dict(self.policy.state_dict())
        self.buffer.clear()

In [6]:
class SACAgent:
    def __init__(self, state_dim, action_dim, lr, gamma, batch_size, buffer_size, is_continuous):
        self.is_continuous = is_continuous
        self.gamma = gamma
        self.batch_size = batch_size
        self.tau = 0.005
        self.alpha = 0.2  # Entropy Coefficient

        # Networks
        self.actor = self._build_actor(state_dim, action_dim).to(device)
        self.q1 = self._build_critic(state_dim, action_dim).to(device)
        self.q2 = self._build_critic(state_dim, action_dim).to(device)
        self.q1_target = self._build_critic(state_dim, action_dim).to(device)
        self.q2_target = self._build_critic(state_dim, action_dim).to(device)
        
        self.q1_target.load_state_dict(self.q1.state_dict())
        self.q2_target.load_state_dict(self.q2.state_dict())

        self.actor_optim = optim.Adam(self.actor.parameters(), lr=lr)
        self.q_optim = optim.Adam(list(self.q1.parameters()) + list(self.q2.parameters()), lr=lr)
        self.replay_buffer = ReplayBuffer(buffer_size, state_dim, action_dim)

    def _build_actor(self, s_dim, a_dim):
        return nn.Sequential(
            nn.Linear(s_dim, 256), nn.ReLU(),
            nn.Linear(256, 256), nn.ReLU(),
            nn.Linear(256, a_dim * 2 if self.is_continuous else a_dim) # Mean+LogStd or Logits
        )

    def _build_critic(self, s_dim, a_dim):
        # Continuous: Input (s, a) -> Output Q(s,a)
        # Discrete: Input (s) -> Output Q(s, .) for all actions
        input_dim = s_dim + a_dim if self.is_continuous else s_dim
        output_dim = 1 if self.is_continuous else a_dim
        return nn.Sequential(
            nn.Linear(input_dim, 256), nn.ReLU(),
            nn.Linear(256, 256), nn.ReLU(),
            nn.Linear(256, output_dim)
        )

    def select_action(self, state):
        with torch.no_grad():
            state = torch.FloatTensor(state).to(device)
            out = self.actor(state)
            
            if self.is_continuous:
                mean, log_std = out.chunk(2, dim=-1)
                std = log_std.clamp(-20, 2).exp()
                dist = Normal(mean, std)
                action = torch.tanh(dist.rsample()) # Reparameterization
                return action.cpu().numpy()
            else:
                probs = F.softmax(out, dim=-1)
                dist = Categorical(probs)
                action = dist.sample()
                return action.item()

    def update(self):
        if len(self.replay_buffer) < self.batch_size: return
        
        # Sample from buffer
        state, action, reward, next_state, done = self.replay_buffer.sample(self.batch_size)
        
        # --- CRITIC UPDATE ---
        with torch.no_grad():
            next_out = self.actor(next_state)
            
            if self.is_continuous:
                # ... (Existing Continuous Logic) ...
                mean, log_std = next_out.chunk(2, dim=-1)
                std = log_std.clamp(-20, 2).exp()
                dist = Normal(mean, std)
                next_action = torch.tanh(dist.rsample())
                log_prob = dist.log_prob(dist.rsample()).sum(-1, keepdim=True)
                log_prob -= (2*(np.log(2) - next_action - F.softplus(-2*next_action))).sum(1, keepdim=True)
                
                q1_next = self.q1_target(torch.cat([next_state, next_action], 1))
                q2_next = self.q2_target(torch.cat([next_state, next_action], 1))
                min_q_next = torch.min(q1_next, q2_next) - self.alpha * log_prob
                target_q = reward + (1 - done) * self.gamma * min_q_next
            else:
                # ... (Discrete Logic) ...
                probs = F.softmax(next_out, dim=-1)
                log_probs = torch.log(probs + 1e-8)
                q1_next = self.q1_target(next_state)
                q2_next = self.q2_target(next_state)
                min_q_next = torch.min(q1_next, q2_next)
                target_v = (probs * (min_q_next - self.alpha * log_probs)).sum(dim=1, keepdim=True)
                target_q = reward + (1 - done) * self.gamma * target_v

        if self.is_continuous:
            current_q1 = self.q1(torch.cat([state, action], 1))
            current_q2 = self.q2(torch.cat([state, action], 1))
        else:
            # === FIX IS HERE ===
            # We must unsqueeze action to match dimensions: [Batch] -> [Batch, 1]
            action_idx = action.long().unsqueeze(1) 
            
            current_q1 = self.q1(state).gather(1, action_idx)
            current_q2 = self.q2(state).gather(1, action_idx)

        critic_loss = F.mse_loss(current_q1, target_q) + F.mse_loss(current_q2, target_q)
        
        self.q_optim.zero_grad()
        critic_loss.backward()
        self.q_optim.step()

        # Actor Update
        curr_out = self.actor(state)
        if self.is_continuous:
            mean, log_std = curr_out.chunk(2, dim=-1)
            std = log_std.clamp(-20, 2).exp()
            dist = Normal(mean, std)
            new_action = torch.tanh(dist.rsample())
            log_prob = dist.log_prob(dist.rsample()).sum(-1, keepdim=True)
            q1_pi = self.q1(torch.cat([state, new_action], 1))
            q2_pi = self.q2(torch.cat([state, new_action], 1))
            actor_loss = (self.alpha * log_prob - torch.min(q1_pi, q2_pi)).mean()
        else:
            probs = F.softmax(curr_out, dim=-1)
            log_probs = torch.log(probs + 1e-8)
            q1_pi = self.q1(state)
            q2_pi = self.q2(state)
            min_q_pi = torch.min(q1_pi, q2_pi)
            actor_loss = (probs * (self.alpha * log_probs - min_q_pi)).sum(dim=1).mean()

        self.actor_optim.zero_grad()
        actor_loss.backward()
        self.actor_optim.step()
        
        # Soft Update Targets
        for p, tp in zip(self.q1.parameters(), self.q1_target.parameters()):
            tp.data.copy_(self.tau * p.data + (1 - self.tau) * tp.data)
        for p, tp in zip(self.q2.parameters(), self.q2_target.parameters()):
            tp.data.copy_(self.tau * p.data + (1 - self.tau) * tp.data)

In [7]:
import matplotlib.pyplot as plt
import seaborn as sns


os.makedirs("images", exist_ok=True)

def evaluate_agent(agent, env_name, model_name, run_name):
    print(f"🎬 Starting Evaluation for {model_name} on {env_name}...")
    
    # 1. Run Testing Loop
    eval_env = gym.make(env_name, render_mode="rgb_array")
    # Video recorder (record episode 0)
    eval_env = gym.wrappers.RecordVideo(
        eval_env, 
        video_folder=f"./videos/test_{run_name}", 
        episode_trigger=lambda x: x == 0, 
        disable_logger=True
    )
    
    test_rewards = []
    test_durations = []
    
    for i in range(100): # 100 Test Episodes
        state, _ = eval_env.reset()
        done = False
        truncated = False
        ep_reward = 0
        steps = 0
        
        while not (done or truncated):
            action = agent.select_action(state) # Deterministic if possible
            state, reward, done, truncated, _ = eval_env.step(action)
            ep_reward += reward
            steps += 1
            
        test_rewards.append(ep_reward)
        test_durations.append(steps)
        
    eval_env.close()

    print("✅ Testing Complete. Generating Stability Plots...", len(test_rewards))
    
    # 2. Generate Stability Plots (Matplotlib)
    # We create a figure with 2 subplots: Reward Distribution and Duration Distribution
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # Plot A: Rewards
    sns.histplot(test_rewards, kde=True, ax=axes[0], color='blue', bins=15)
    axes[0].set_title(f"{run_name} - Reward Stability (N=100)")
    axes[0].set_xlabel("Episode Reward")
    axes[0].set_ylabel("Frequency")
    
    # Plot B: Durations (Stability Metric)
    sns.histplot(test_durations, kde=True, ax=axes[1], color='green', bins=15)
    axes[1].set_title(f"{run_name} - Duration Stability (N=100)")
    axes[1].set_xlabel("Episode Duration (Steps)")
    axes[1].set_ylabel("Frequency")
    
    plt.tight_layout()
    
    # Save locally for your report
    plot_filename = f"images/stability_{run_name}.png"
    plt.savefig(plot_filename)
    plt.close() # Close plot to prevent double printing in notebook
    
    # 3. Log Metrics & Images to WandB
    wandb.log({
        "test/avg_reward": np.mean(test_rewards),
        "test/std_reward": np.std(test_rewards),
        "test/avg_duration": np.mean(test_durations),
        "test/std_duration": np.std(test_durations),
        # Log the actual image we just created
        "stability_analysis": wandb.Image(plot_filename, caption=f"Stability Analysis: {run_name}")
    })
    
    print(f"✅ Eval Complete. Plot saved to {plot_filename}")

In [8]:
def make_env(env_name):
    """Wrapper to handle video recording trigger"""
    env = gym.make(env_name, render_mode="rgb_array")
    return env

def train_agent():
    with wandb.init():
        config = wandb.config

        run_name_str = f"{config.model_name}_LR{config.learning_rate}_G{config.gamma}_BS{config.batch_size}_RMS{config.replay_memory_size}"
        wandb.run.name = run_name_str
        
        env_name = config.env_name
        model_name = config.model_name
        
        # Create Environment and Video Recorder
        env = gym.make(env_name, render_mode="rgb_array")
        env = gym.wrappers.RecordVideo(
            env, 
            video_folder=f"./videos/{run_name_str}_{wandb.run.id}", 
            episode_trigger=lambda e: e % 249 == 0, # Record every quarter
            disable_logger=True
        )
        
        state_dim = env.observation_space.shape[0]
        is_continuous = isinstance(env.action_space, gym.spaces.Box)
        if not is_continuous:
            # Fallback: Check if it has a shape tuple (Box has shape, Discrete usually doesn't)
            if hasattr(env.action_space, 'shape') and len(env.action_space.shape) > 0:
                is_continuous = True
        action_dim = env.action_space.shape[0] if is_continuous else env.action_space.n
        
        print(f"🚀 Training {model_name} on {env_name} (Continuous: {is_continuous})")

        # Initialize Model
        if model_name == "PPO":
            agent = PPOAgent(state_dim, action_dim, config.learning_rate, config.gamma, 
                             k_epochs=4, eps_clip=0.2, is_continuous=is_continuous)
        elif model_name == "A2C":
            agent = A2CAgent(state_dim, action_dim, config.learning_rate, config.gamma, 
                             k_epochs=1, eps_clip=0.0, is_continuous=is_continuous)
        elif model_name == "SAC":
            agent = SACAgent(state_dim, action_dim, config.learning_rate, config.gamma, 
                             config.batch_size, config.replay_memory_size, is_continuous)

        # Training Loop
        total_steps = 0
        
        for episode in range(500):
            state, _ = env.reset()
            episode_reward = 0
            
            for t in range(500): # Max steps per episode
                if model_name == "SAC":
                    action = agent.select_action(state)
                    # Handle Gym step
                    next_state, reward, terminated, truncated, _ = env.step(action)
                    done = terminated or truncated
                    
                    # Store & Update (Off-Policy)
                    agent.replay_buffer.push(state, action, reward, next_state, done)
                    agent.update()
                else:
                    action = agent.select_action(state)
                    next_state, reward, terminated, truncated, _ = env.step(action)
                    done = terminated or truncated
                    
                    # Store (On-Policy)
                    agent.buffer.rewards.append(reward)
                    agent.buffer.dones.append(done)
                
                state = next_state
                episode_reward += reward
                total_steps += 1
                
                if done: break
            
            # Update On-Policy Agents
            if model_name in ["PPO", "A2C"]:
                agent.update()
            
            wandb.log({"train/reward": episode_reward, "total_steps": total_steps})
                
        env.close()

        # --- 4. SAVE MODEL ---
        # Save the model weights locally and to WandB
        model_path = f"models/{run_name_str}.pth"
        if model_name == "SAC":
            torch.save(agent.actor.state_dict(), model_path)
        else:
            torch.save(agent.policy.state_dict(), model_path)
        artifact = wandb.Artifact(
            name=f"model-{wandb.run.id}", 
            type="model",
            description=f"Model for {run_name_str}"
        )

        # Add your file to the artifact
        artifact.add_file(model_path)

        # Log the artifact to WandB
        wandb.log_artifact(artifact)

        # --- 5. EVALUATION PHASE ---
        # Run 100 tests and log duration [cite: 1703]
        evaluate_agent(agent, env_name, model_name, run_name_str)

In [ ]:
# 1. Login to WandB
wandb.login(key="YOUR_KEY")

models = ['SAC']

for model in models:
    sweep_configuration = {
        'name': 'SAC Bayes - 500',
        'method': 'bayes',
        'metric': {'name': 'test/avg_reward', 'goal': 'maximize'},
        'parameters': {
            'env_name': {'values': ['CartPole-v1']},
            'model_name': {'values': [model]},
            'learning_rate': {'values': [1e-3, 3e-4, 1e-4]},
            'gamma': {'values': [0.99, 0.95]}, # Discount Factor [cite: 6]
            'batch_size': {'values': [64, 128, 256]}, # Learning Batch Size [cite: 10]
            'replay_memory_size': {'values': [10000, 50000]}, # Replay Memory Size [cite: 9]
        },
        'run_cap': 14
    }
    
    # 2. Initialize Sweep
    sweep_id = wandb.sweep(sweep_configuration, project="Local_RL_Assignment3_CartPole_Grid_with_Test")
    
    # 3. Run Agent (Execute this cell to start training)
    wandb.agent(sweep_id, function=train_agent)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\user\_netrc
wandb: Currently logged in as: abdelrahmanayman2003 (abdelrahmanayman2003-cairo-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Create sweep with ID: cnm8vmat
Sweep URL: https://wandb.ai/abdelrahmanayman2003-cairo-university/Local_RL_Assignment3_CartPole_Grid_with_Test/sweeps/cnm8vmat


wandb: Agent Starting Run: def9v3ch with config:
wandb: 	batch_size: 128
wandb: 	env_name: CartPole-v1
wandb: 	gamma: 0.95
wandb: 	learning_rate: 0.0001
wandb: 	model_name: SAC
wandb: 	replay_memory_size: 10000


🚀 Training SAC on CartPole-v1 (Continuous: False)
🎬 Starting Evaluation for SAC on CartPole-v1...
✅ Testing Complete. Generating Stability Plots... 100
✅ Eval Complete. Plot saved to images/stability_SAC_LR0.0001_G0.95_BS128_RMS10000.png


test/avg_duration,▁
test/avg_reward,▁
test/std_duration,▁
test/std_reward,▁
total_steps,▁▁▁▁▁▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▄▅▅▅▅▅▆▆▇▇▇▇▇▇▇▇▇███
train/reward,▁▁▁▁▃▄▄▄▆▅▅▆▄▃▄▃▃▅▃█▆▅▅▅█▄▄▄▄▄▄▇▄▄▆▅▅▄▅▅
test/avg_duration,231.96
test/avg_reward,231.96
test/std_duration,33.8712
test/std_reward,33.8712
total_steps,117630


...

c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\gymnasium\wrappers\rendering.py:293: UserWarning: WARN: Overwriting existing videos at d:\Abdelrahman files\University\Credit\Fall 2025\RL\Assignments\Assignment 3\Local\videos\test_SAC_LR0.0001_G0.99_BS256_RMS50000 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


✅ Testing Complete. Generating Stability Plots... 100
✅ Eval Complete. Plot saved to images/stability_SAC_LR0.0001_G0.99_BS256_RMS50000.png


test/avg_duration,▁
test/avg_reward,▁
test/std_duration,▁
test/std_reward,▁
total_steps,▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▅▅▅▅▅▆▆▇▇▇▇█
train/reward,▁▁▁▁▂▃█▅▄▅▅▅▄▄▄▄▅▅▅▅▆███████████████████
test/avg_duration,500
test/avg_reward,500
test/std_duration,0
test/std_reward,0
total_steps,164886


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 4f8vnrej with config:
wandb: 	batch_size: 256
wandb: 	env_name: CartPole-v1
wandb: 	gamma: 0.99
wandb: 	learning_rate: 0.0001
wandb: 	model_name: SAC
wandb: 	replay_memory_size: 50000


🚀 Training SAC on CartPole-v1 (Continuous: False)
